## **Custom CNN**

This notebook documents the creation and training of the custom Convolutional Neural Network (CNN). This serves as the baseline model against which the performance of the transfer learning models will be compared.

Goal: Train a Custom CNN using 80/10/10 stratified split, 0-1 normalization, data augmentation, and class weights (due to class imbalance).

## Project Setup and Initialization

Imports and Paths

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import os

In [2]:
# Import utility functions from  uploaded files
from data_utils import perform_stratified_split, DataGeneratorUtils, TARGET_SIZE, SEED, BATCH_SIZE
from train_utils import compile_model, get_callbacks

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Path to your metadata.csv file
METADATA_PATH = "/content/drive/MyDrive/NOVA_IMS/Deep_Learning_Project/rare_species/metadata.csv"
# Root directory containing all your original, un-split image files
IMAGE_ROOT_DIR = "/content/drive/MyDrive/NOVA_IMS/Deep_Learning_Project/rare_species"
# The target folder where the stratified data structure (train/val/test) will be created
DATA_TARGET_DIR = "/content/drive/MyDrive/NOVA_IMS/Deep_Learning_Project/data"

# Create the results directory if it doesn't exist to save model weights
os.makedirs("/content/drive/MyDrive/NOVA_IMS/Deep_Learning_Project/outputs", exist_ok=True)

Data Splitting and Generator Creation

*We use the functions from data_utils.py to handle the reproducible split and the data pipeline creation.*

In [5]:
# Load the metadata file
try:
    metadata_df = pd.read_csv(METADATA_PATH)
except FileNotFoundError:
    print("ERROR: Metadata file not found.")

In [29]:
# IMPORTANT: RUN perform_stratified_split ONLY ONCE!
#data_base_path = perform_stratified_split(metadata_df, IMAGE_ROOT_DIR, DATA_TARGET_DIR)
#print(f"Data structure created/verified at: {data_base_path}")

Data Split: Train=9585, Validation=1199, Test=1199
Organizing train set...
Organizing validation set...
Organizing test set...
Data directory structure successfully created/updated.
Data structure created/verified at: /content/drive/MyDrive/NOVA_IMS/Deep_Learning_Project/data


In [28]:
#!ls -R /content/drive/MyDrive/NOVA_IMS/Deep_Learning_Project/data/

Verification: counting images per class in split directories

In [6]:
def count_images_per_class(base_directory, set_name):
    """Counts the number of images in each class (family folder) for a given set."""
    directory = os.path.join(base_directory, set_name)
    class_counts = {}

    if not os.path.exists(directory):
        return {f"ERROR: Directory not found at {directory}": 0}

    for folder in os.listdir(directory):
        folder_path = os.path.join(directory, folder)
        if os.path.isdir(folder_path):
            # Count files in the class folder
            image_count = len([f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
            class_counts[folder] = image_count
    return class_counts

In [30]:
# Count images in train, validation, and test directories
train_class_counts = count_images_per_class(DATA_TARGET_DIR, 'train')
val_class_counts = count_images_per_class(DATA_TARGET_DIR, 'validation')
test_class_counts = count_images_per_class(DATA_TARGET_DIR, 'test')

# Display results
print("\n--- Class Counts Verification ---")
print("Number of images per class in the TRAIN directory (Top 5):")
# Sort and print for readability
print(pd.Series(train_class_counts).sort_values(ascending=False).head(5))

print("\nNumber of images per class in the VALIDATION directory (Top 5):")
print(pd.Series(val_class_counts).sort_values(ascending=False).head(5))

print("\nNumber of images per class in the TEST directory (Top 5):")
print(pd.Series(test_class_counts).sort_values(ascending=False).head(5))

# Check for overall size consistency
total_counted = sum(train_class_counts.values()) + sum(val_class_counts.values()) + sum(test_class_counts.values())
print(f"\nTotal images successfully counted across all splits: {total_counted}")


--- Class Counts Verification ---
Number of images per class in the TRAIN directory (Top 5):
cercopithecidae    240
dactyloidae        240
formicidae         233
plethodontidae     216
carcharhinidae     216
dtype: int64

Number of images per class in the VALIDATION directory (Top 5):
cercopithecidae    30
dactyloidae        30
formicidae         29
plethodontidae     27
carcharhinidae     27
dtype: int64

Number of images per class in the TEST directory (Top 5):
dactyloidae        30
cercopithecidae    30
formicidae         29
plethodontidae     27
salamandridae      27
dtype: int64

Total images successfully counted across all splits: 11983


In [7]:
data_base_path = DATA_TARGET_DIR


In [8]:
#  Initialize Data Generators
data_util = DataGeneratorUtils(data_base_path)

train_generator = data_util.create_generators('train')
val_generator = data_util.create_generators('validation')


Found 9585 images belonging to 202 classes.
Found 1199 images belonging to 202 classes.


In [33]:
NUM_CLASSES = train_generator.num_classes

In [34]:
# Calculate Class Weights (Crucial for rare species imbalance)
class_weights = data_util.calculate_class_weights(train_generator)

print(f"\nSetup complete. Total classes: {NUM_CLASSES}")
print(f"Train samples: {train_generator.samples}, Validation samples: {val_generator.samples}")

Class weights calculated for 202 classes.

Setup complete. Total classes: 202
Train samples: 9585, Validation samples: 1199


In [9]:
import os

# Define the location of the directory you want to archive (the source)
# This path contains your 'train', 'validation', and 'test' folders.
SOURCE_DATA_DIR = "/content/drive/MyDrive/NOVA_IMS/Deep_Learning_Project/data"

# Define the desired path and name for the output ZIP file
# The file will be placed in the same project directory on your Drive.
OUTPUT_ZIP_PATH = "/content/drive/MyDrive/NOVA_IMS/Deep_Learning_Project/data.zip"

print(f"Creating archive from: {SOURCE_DATA_DIR}")

# The command below zips the 'data' directory.
# -r: recursively include directories
# -q: quiet mode (less output)
# Note: We must navigate to the parent directory first to zip the 'data' folder itself.

# Navigate to the parent directory of 'data' (which is 'Deep_Learning_Project')
os.chdir(os.path.dirname(SOURCE_DATA_DIR))

# Execute the zip command
!zip -r -q {OUTPUT_ZIP_PATH} data/

print("\n--- Zipping Complete ---")
print(f"Archive created at: {OUTPUT_ZIP_PATH}")

# Navigate back to a safe location
os.chdir('/content')

Creating archive from: /content/drive/MyDrive/NOVA_IMS/Deep_Learning_Project/data


zip error: Interrupted (aborting)

--- Zipping Complete ---
Archive created at: /content/drive/MyDrive/NOVA_IMS/Deep_Learning_Project/data.zip


In [10]:
import os
import shutil

In [11]:
# Navigate back to a safe location
os.chdir('/content')

In [12]:
# Source ZIP file on Google Drive (where you just saved it)
GDRIVE_ZIP_PATH = "/content/drive/MyDrive/NOVA_IMS/Deep_Learning_Project/data.zip"

# Destination folder for the unzipped data on the local Colab disk
LOCAL_DATA_DIR = "/content/data"
LOCAL_ZIP_PATH = "/content/data.zip"

# Create the local directory
os.makedirs(LOCAL_DATA_DIR, exist_ok=True)

In [13]:
#  Copy and Unzip
print("Copying data from Google Drive to local disk...")
# Copy the single zip file (this is fast)
shutil.copy(GDRIVE_ZIP_PATH, LOCAL_ZIP_PATH)

# Unzip the file onto the local disk
print("Unzipping data...")
# -q makes it quiet, -d sets the destination folder
!unzip -q {LOCAL_ZIP_PATH} -d {LOCAL_DATA_DIR}

print("\nData transfer complete")

Copying data from Google Drive to local disk...
Unzipping data...

Data transfer complete


In [14]:
# Re-initialize Generators with Local Path

# Point the base path to the directory containing 'train', 'validation', and 'test'.
FAST_I_O_BASE_PATH = os.path.join(LOCAL_DATA_DIR, 'data')

print(f"\nRe-initializing generators using the faster local path: {FAST_I_O_BASE_PATH}")

# Re-initialize Data Generators with the fast, local path
data_util = DataGeneratorUtils(FAST_I_O_BASE_PATH)

train_generator = data_util.create_generators('train')
val_generator = data_util.create_generators('validation')

# Recalculate class weights (important to run after generator initialization)
class_weights = data_util.calculate_class_weights(train_generator)

NUM_CLASSES = train_generator.num_classes
print(f"\nGenerators are ready. Total classes: {NUM_CLASSES}")


Re-initializing generators using the faster local path: /content/data/data
Found 9585 images belonging to 202 classes.
Found 1199 images belonging to 202 classes.
Class weights calculated for 202 classes.

Generators are ready. Total classes: 202


# Hyperparameter Tuner and Callbacks

In [15]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [17]:
!pip install keras_tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 6.6 MB/s eta 0:00:00


In [18]:
import keras_tuner as kt

In [19]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

In [20]:
PROJECT_NAME = 'custom_cnn_hyperband'

In [21]:
# Early Stopping: Stops training if val_loss doesn't improve for 5 epochs
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

In [22]:
# Reduce LR on Plateau: Reduces LR if val_loss doesn't improve for 5 epochs
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=5,
    min_lr=1e-6,
    verbose=1
)

In [23]:
callbacks = [early_stopping, reduce_lr]

In [24]:
def build_hypermodel(hp):
    """
    Builds a Keras Sequential model parameterized by KerasTuner's HyperParameters (hp).
    Input shape is now (128, 128, 3).
    """
    model = Sequential()

    # Tunable Hyperparameters
    hp_filters_1 = hp.Int('filters_1', min_value=32, max_value=128, step=32)
    hp_dropout = hp.Float('dropout', min_value=0.2, max_value=0.5, step=0.1)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-3, 1e-4, 5e-5])

    # Model Architecture
    model.add(Conv2D(hp_filters_1, (3, 3), activation='relu',
                     input_shape=(TARGET_SIZE[0], TARGET_SIZE[1], 3)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Classification Head
    model.add(Flatten())
    model.add(Dropout(hp_dropout))
    model.add(Dense(NUM_CLASSES, activation='softmax'))

    # Compile Model
    model.compile(
        optimizer=Adam(learning_rate=hp_learning_rate),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [57]:
#Instantiate the Hyperband Tuner
tuner = kt.Hyperband(
    hypermodel=build_hypermodel,
    objective='val_accuracy', # Maximize validation accuracy
    max_epochs=30,           # Max epochs for a full training run
    factor=3,                # Halving factor for Hyperband
    directory='/content/drive/MyDrive/NOVA_IMS/Deep_Learning_Project/outputs', # Path to save results
    project_name=PROJECT_NAME,
    overwrite=True           # Overwrite previous search results
)

print("Tuner search space summary:")
tuner.search_space_summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Tuner search space summary:
Search space summary
Default search space size: 3
filters_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 32, 'sampling': 'linear'}
dropout (Float)
{'default': 0.2, 'conditions': [], 'min_value': 0.2, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.001, 'conditions': [], 'values': [0.001, 0.0001, 5e-05], 'ordered': True}


In [58]:
# Run the search using the training and validation generators and class weights
tuner.search(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    callbacks=callbacks,
    class_weight=class_weights,
    verbose=1
)


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
96                |96                |filters_1
0.2               |0.2               |dropout
0.001             |0.001             |learning_rate
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
3                 |3                 |tuner/bracket
0                 |0                 |tuner/round



/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/2
227/300 ━━━━━━━━━━━━━━━━━━━━ 13:51 11s/step - accuracy: 0.0130 - loss: 133.1130

KeyboardInterrupt: 

took too long

In [25]:
from sklearn.model_selection import train_test_split

In [26]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [27]:
# Path to the training directory on the fast local disk
TRAIN_ROOT = "/content/data/data/train"
SUBSET_RATIO = 0.20 # Use 20% of the data for tuning

In [28]:
# Gather all file paths and labels
file_paths = []
labels = []
for family_class in os.listdir(TRAIN_ROOT):
    family_path = os.path.join(TRAIN_ROOT, family_class)
    if os.path.isdir(family_path):
        for img_file in os.listdir(family_path):
            # We save a relative path that flow_from_dataframe can use
            file_paths.append(os.path.join(family_class, img_file))
            labels.append(family_class)

full_train_df = pd.DataFrame({'filename': file_paths, 'class': labels})


In [29]:
#  Perform Stratified Split for the Subset
# Stratified sampling ensures all 202 classes are present in the 20% subset.
_, subset_df = train_test_split(
    full_train_df,
    test_size=SUBSET_RATIO,
    random_state=SEED,
    stratify=full_train_df['class']
)


In [30]:
# Define the subset generator
subset_datagen = ImageDataGenerator(
    rescale=1./255, # 0-1 Normalization
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


In [33]:
# Create the final subset generator
subset_generator = subset_datagen.flow_from_dataframe(
    subset_df,
    directory=TRAIN_ROOT, # The absolute root where the image folders are located
    x_col='filename',
    y_col='class',
    target_size=TARGET_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    seed=SEED
)

Found 1917 validated image filenames belonging to 202 classes.


In [34]:

print(f"Original training samples: {len(full_train_df)}, Tuning subset samples: {len(subset_df)}")
print(f"New steps per epoch for tuning: {np.ceil(len(subset_df) / BATCH_SIZE)}")

Original training samples: 9585, Tuning subset samples: 1917
New steps per epoch for tuning: 30.0


In [35]:
PROJECT_NAME = 'custom_cnn_hyperband_fastest'

In [36]:
tuner = kt.Hyperband(
    hypermodel=build_hypermodel,
    objective='val_accuracy',
    max_epochs=15,           # Budget sufficient for proxy tuning
    factor=3,
    directory='/content/drive/MyDrive/NOVA_IMS/Deep_Learning_Project/outputs',
    project_name=PROJECT_NAME,
    overwrite=True
)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [37]:
tuner.search(
    subset_generator,
    epochs=15,
    validation_data=val_generator, # IMPORTANT: Use the full val_generator!
    callbacks=callbacks,
    class_weight=class_weights,
    verbose=1
)

Trial 30 Complete [00h 07m 51s]
val_accuracy: 0.01668056659400463

Best val_accuracy So Far: 0.01668056659400463
Total elapsed time: 02h 44m 48s
